In [1]:
import pandas as pd
import numpy as np
import scipy as sp

In [2]:
import seaborn as sns
from matplotlib import pyplot as plt

In [139]:
class UGMM(object):
    
    def __init__(self, X, K=2):
        self.X = X
        self.K = K
        self.N = self.X.shape[0]
        #Instantiation with components passed by an user and no. of data points inferred.

    def _init(self):
#         self.alpha=np.random.rand(self.K,1)
        #Declaring priors on parameters of distribution
        self.m_prior= np.zeros((self.K,1))
        #Means for distribution of mean of gaussian components all assumed to be 0
        self.m=self.m_prior.copy()
        self.beta_prior=np.random.rand(self.K,1)
        #Beta for Gaussian random
        self.beta=self.beta_prior.copy()
        self.nu_prior=np.ones((self.K,1))*np.random.randint(1,10)
        #random shape parameter for Wishart/Gamma
        self.nu= self.nu_prior.copy()
        self.W_prior=np.ones((self.K,1))*np.random.randint(1,10)
        #random scale parameter for Wishart/Gamma
        self.W=self.W_prior.copy()
        self.alpha_prior=np.ones((self.K,1))*1
        #Same alpha parameter for Dirichlet distribution for every gaussian component
        self.alpha=self.alpha_prior.copy()
        self.r=np.zeros((self.N,self.K))
        #Intitalization for responsibity matrix and other E-M step variables
        self.x_nebla=np.zeros((self.K,1))
        self.Nk=np.zeros((self.K,1))
        self.S=np.zeros((self.K,1))
        self.pi_nebla=np.zeros((self.K,1))
        self.lamda_nebla=np.zeros((self.K,1))
        self.E_mu_lamda=np.zeros((self.N,self.K))
        
        

        print('Init mean')
        print(self.m)
        

    def E_Step(self):
        #Equivalent E-Step Calulations
        for n in range(self.N):
            for k in range(self.K):
                self.pi_nebla[k]=np.exp(sp.special.digamma(self.alpha[k])-sp.special.digamma(self.alpha.sum()))
                self.lamda_nebla[k]=np.exp(sp.special.digamma(self.nu[k]/2, out=None)+np.log(2)+np.log(self.W[k]))
                self.E_mu_lamda[n][k]=(self.beta[k]**(-1))+self.nu[k]*self.W[k]*(self.X[n]-self.m[k])**2
                self.r[n][k]=self.pi_nebla[k]*(self.lamda_nebla[k]**0.5)*(np.exp(-0.5*self.E_mu_lamda[n][k]))
        row_sums = self.r.sum(axis=1)
        self.r= self.r / row_sums[:, np.newaxis]
        
        
    def fit(self, max_iter=10000, tol=1e-12):
        #Intialization of priors
        self._init()
        self.m_history = [self.m.copy()]
        #Perform E-step
        self.E_Step()
        for iter_ in range(1, max_iter+1):
            #perform M-step
            self.M_Step()
            self.m_history.append(self.m.copy())
            if iter_ % 25 == 0:
                print(iter_, self.m_history[iter_])
                
            if np.linalg.norm(self.m_history[-1]-self.m_history[-2],2) <= tol:
                print('VI converged with ll %.3f at iteration %d'%(self.m_history[-1][0],iter_))
                break
            #Perform E-step
            self.E_Step()
        if iter_ == max_iter:
            print('VI ended with ll %.3f'%(self.m_history[-1][0]))


    def M_Step(self):
        self._update_Nk()
        self._update_x_nebla()
        self._update_S()
        self._update_alpha()
        self._update_W()
        self._update_beta()
        self._update_nu()
        self._update_m()

    def _update_Nk(self):
        for k in range(self.K):
            self.Nk[k]=self.r[:,k].sum()
            
    def _update_x_nebla(self):
        for k in range(self.K):
            self.x_nebla[k]=0
            for n in range(self.N):
#                 print(self.Nk)
                self.x_nebla[k]+=self.r[n][k]*self.X[n]/self.Nk[k]
    
    def _update_S(self):
        for k in range(self.K):
            self.S[k]=0
            for n in range(self.N):
                self.S[k]+=(self.r[n][k]*(self.X[n]-self.x_nebla[k])**(2))/self.Nk[k]
                
    def _update_alpha(self):
        for k in range(self.K):
            #Should the first term on RHS be the previous value of alpha? Should priors everywhere be replaced with previous value?
            self.alpha[k]=self.alpha_prior[k]+self.Nk[k]
            
    def _update_beta(self):
        for k in range(self.K):
            self.beta[k]=self.beta_prior[k]+self.Nk[k]
        
    def _update_m(self):
        for k in range(self.K):
            self.m[k]=(self.beta_prior[k]*self.m_prior[k]+self.Nk[k]*self.x_nebla[k])/self.beta[k]
    
    
    def _update_W(self):
        for k in range(self.K):
            self.W[k]=((1/self.W_prior[k])+self.Nk[k]*self.S[k]+(self.beta_prior[k]*self.Nk[k]/(self.beta_prior[k]+self.Nk[k]))*(self.x_nebla[k]-self.m_prior[k])**2)**(-1)
        
    
    def _update_nu(self):
        for k in range(self.K):
            self.nu[k]=self.nu_prior[k]+self.Nk[k]
    
#     def _calculate_rnk
        

In [140]:
data=pd.read_csv('data2.txt',header=None)

In [141]:
X =np.array(data.iloc[0,0])
for i in range(1,1000):
    X= np.append(X, data.iloc[i,0])
    

In [149]:
ugmm = UGMM(X, 2)
ugmm.fit()

Init mean
[[0.]
 [0.]]
25 [[0.45163184]
 [0.45213982]]
50 [[0.45135009]
 [0.45224886]]
75 [[0.45095999]
 [0.45239414]]
100 [[0.45043146]
 [0.45258348]]
125 [[0.44972429]
 [0.45282705]]
150 [[0.44878265]
 [0.45313876]]
175 [[0.44752497]
 [0.45353885]]
200 [[0.4458243]
 [0.454059 ]]
225 [[0.44346672]
 [0.45475331]]
250 [[0.440052  ]
 [0.45572462]]
275 [[0.43471411]
 [0.45719955]]
300 [[0.42510172]
 [0.45980616]]
325 [[0.40140204]
 [0.4662584 ]]
350 [[0.29826806]
 [0.50533112]]
375 [[0.29692752]
 [0.54268084]]
400 [[0.30569579]
 [0.5566557 ]]
425 [[0.30977849]
 [0.5623701 ]]
450 [[0.31153309]
 [0.5647175 ]]
475 [[0.31226952]
 [0.56568567]]
500 [[0.31257599]
 [0.56608575]]
525 [[0.3127031 ]
 [0.56625121]]
550 [[0.31275575]
 [0.56631966]]
575 [[0.31277755]
 [0.56634798]]
600 [[0.31278657]
 [0.5663597 ]]
625 [[0.3127903 ]
 [0.56636455]]
650 [[0.31279184]
 [0.56636656]]
675 [[0.31279248]
 [0.56636739]]
700 [[0.31279275]
 [0.56636773]]
725 [[0.31279286]
 [0.56636788]]
750 [[0.3127929 ]
 [0.566

In [146]:
mean=ugmm.m

In [147]:
mean

array([[0.56625115],
       [0.31186257]])